
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/SemEval2016-Task6-raw-annotations-stance.csv")
#df=df1.ffill()
df=df1
df

,Worker ID,Instance ID,Target,Tweet,Stance,Opinion towards
0,worker_1,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,TARGET
1,worker_2,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,FAVOR,TARGET
2,worker_3,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,OTHER
3,worker_4,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,OTHER
4,worker_5,2370.0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,AGAINST,TARGET
...,...,...,...,...,...,...
53094,worker_113,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET
53095,worker_85,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,OTHER
53096,worker_56,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET
53097,worker_84,10951.0,Hillary Clinton,@TheAtlantic cause #Hillary Clinton is a LYIN...,AGAINST,TARGET


In [ ]:
unique_labels = df['Stance'].unique()
print(unique_labels)

['AGAINST' 'FAVOR' 'NONE' 'NEUTRAL']


In [ ]:
def data_preprocessing(df):
  encoder = LabelEncoder()
  df['Stance'] = encoder.fit_transform(df['Stance'])
  from sklearn.utils import resample
  
  # Separate the majority and minority classes
  df_majority = df[df['Stance'] == 0]
  df_minority = df[df['Stance'] == 1]

  # Resample the majority class to match the number of samples in the minority class
  df_majority_resampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

  # Concatenate the resampled majority class with the minority class
  df_resampled = pd.concat([df_majority_resampled, df_minority])

  # Shuffle the rows of the resampled dataset
  df_resampled = df_resampled.sample(frac=1, random_state=42)
  df = df_resampled
  def clean_data(review):
      review = re.sub('[^a-zA-Z]', ' ', review)
      review = review.lower()
      return review

  df['Tweet'] = df['Tweet'].apply(clean_data)

  def remove_stop_words(review):
      review_minus_sw = []
      stop_words = stopwords.words('english')
      review = review.split()
      review = [review_minus_sw.append(word) for word in review if word not in stop_words]
      review = ' '.join(review_minus_sw)
      return review

  df['Tweet'] = df['Tweet'].apply(remove_stop_words)

  def lematize(review):
      lemmatizer = WordNetLemmatizer()
      review = review.split()
      review = [lemmatizer.lemmatize(word) for word in review]
      review = ' '.join(review)
      return review
  df['Tweet'] = df['Tweet'].apply(lematize)

  message = df['Tweet']
  stance = df['Stance'] 
  #splitting train and test data
  x_train, x_test , y_train, y_test = train_test_split(message, stance , test_size = 0.20)

  # model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)
  vocab_size = 10000
  X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in x_train]
  X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in x_test]
 # X_train, X_test , y_train, y_test = train_test_split(input_data, stance, test_size=0.20)

  #apply padding for each sentence have same size 
  max_length = 100
  X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
  X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

  

  def define_rnn_model():
     model = Sequential()
     model.add(Embedding(vocab_size, 100, input_length=max_length))
     model.add(LSTM(128))
     model.add(Dense(1, activation='sigmoid'))
     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
     return model

  # call the define_rnn_model function to create the model object
  model = define_rnn_model()
  # print the model summary
  model.summary()
  #calling adam optimizer
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
   
  #visualizing each eppoch
  history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
  #caluclating loss and accuracy
  loss, accuracy = model.evaluate(X_test,y_test)

  #printing accuracy
  print('Testing Accuracy is {} '.format(accuracy*100))
    
  y_prob = model.predict(X_test)
  y_predict = (y_prob>=0.5).astype(int)
  from sklearn.metrics import classification_report
  print(classification_report(y_test,y_predict,target_names = ["Against","Favor"]))

In [ ]:
df_Feminist_Movement= df[df['Target'] == 'Feminist Movement']
df_Hillary_Clinton= df[df['Target'] == 'Hillary Clinton']
df_Legalization_of_Abortion= df[df['Target'] == 'Legalization of Abortion']
df_Atheism= df[df['Target'] == 'Atheism']
df_Donald_Trump= df[df['Target'] == 'Donald Trump']
targets=[df_Feminist_Movement,df_Hillary_Clinton,df_Legalization_of_Abortion,df_Atheism,df_Donald_Trump]
for i in range(5):
  print("*************",targets[i],"******************")
  data_preprocessing(targets[i])

<ipython-input-8-4721b0f5f405>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Stance'] = encoder.fit_transform(df['Stance'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
177/177 [==============================] - 36s 192ms/step - loss: 0.6940 - acc: 0.4802 - val_loss: 0.6934 - val_acc: 0.4915
Epoch 2/10
177/177 [==============================] - 36s 201ms/step - loss: 0.6933 - acc: 0.5053 - val_loss: 0.6934 - val_acc: 0.4915
Epoch 3/10
177/177 [==================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-8-4721b0f5f405>:3: SettingWithCopyWarning: 
A value 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
114/114 [==============================] - 27s 205ms/step - loss: 0.6938 - acc: 0.5008 - val_loss: 0.6932 - val_acc: 0.4967
Epoch 2/10
114/114 [==============================] - 25s 223ms/step - loss: 0.6934 - acc: 0.5041 - val_loss: 0.6932 - val_acc: 0.5033
Epoch 3/10
114/114 [==================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-8-4721b0f5f405>:3: SettingWithCopyWarning: 
A value 

...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-8-4721b0f5f405>:3: SettingWithCopyWarning: 
A value 

              precision    recall  f1-score   support

     Against       0.50      1.00      0.67       430
       Favor       0.00      0.00      0.00       428

    accuracy                           0.50       858
   macro avg       0.25      0.50      0.33       858
weighted avg       0.25      0.50      0.33       858

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_4 (LSTM)               (None, 128)               117248    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_______________________________________________

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-8-4721b0f5f405>:3: SettingWithCopyWarning: 
A value 

              precision    recall  f1-score   support

     Against       0.00      0.00      0.00       292
       Favor       0.50      1.00      0.67       295

    accuracy                           0.50       587
   macro avg       0.25      0.50      0.33       587
weighted avg       0.25      0.50      0.34       587

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_5 (LSTM)               (None, 128)               117248    
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_______________________________________________

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data_preprocessing(df)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_6 (LSTM)               (None, 128)               117248    
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
578/578 [==============================] - 121s 203ms/step - loss: 0.6936 - acc: 0.4940 - val_loss: 0.6932 - val_acc: 0.5010
Epoch 2/10
578/578 [==============================] - 113s 196ms/step - loss: 0.6932 - acc: 0.5008 - val_loss: 0.6931 - val_acc: 0.5010
Epoch 3/10
578/578 [================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
